# Guía Streamlit en Colab

[Streamlit](https://streamlit.io/) es una librería destinada a crear y compartir aplicaciones o dashboards. Su objetivo es ser una herramienta fácil de usar que permita ejecutar scripts directamente y desplegarlos en un aplicativo web.\
Sin embargo, antes de ver su funcionamiento, debemos tener en cuenta que, si bien Streamlit permite correr el aplicativo, debemos de tener una manera de alojarlo y "hacerlo visible" a la red. Existen diferentes métodos que cumplen dicho objetivo, también dependiendo del nivel de madurez que se requiera, bien sea simplemente testeo o producción; una alternativa eficaz, rápida y que puede ser fácilmente escalable es [Ngrok](https://ngrok.com/).

Ngrok es un servicio y aplicativo globalmente distribuido que asegura, acelera y protege las aplicaciones y servicios de red. En este caso, nos servirá para crear el tunel http que nos deje visualizar el dashboard corriendo con Streamlit desde el entorno del cuadernillo o script de Python.\
El servicio gratuito de Ngrok, tras crear la cuenta tiene la posibilidad de mantener un agente activo simultáneo, cuya configuración está dada por el token de autenticación único por usuario y túnel.

In [ ]:
!pip install streamlit -q
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00


In [ ]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile

In [ ]:
"""
%%writefile app.py
import streamlit as st

st.write('# Hello World ')
st.write('## Run Streamlit on Colab with `pyngrok` ')
"""

Overwriting app.py


Como al momento de correr el script con Streamlit se requiere del archivo .py donde se encuentre el código, utilizamos %%writefile para crearlo. Posteriormente hacemos toda la configuración de Ngrok con el token y demás datos de protocolo.

In [ ]:
token = '2jkoybOVU1P69URUTrdHkZ9gzPe_7xRntA99LEVA6TiNKUsxi'

In [ ]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run detector.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://04d1-34-145-150-66.ngrok-free.app


In [ ]:
#exit("Stopping the execution")

## Cómo usar Streamlit

Dado que lo que busca streamlit es ser lo más sencillo posible de usar, su funcionamiento es bastante simple. Las llamadas a las funciones de streamlit no son para nada complicadas y son fácilmente modificables para usarlas de la manera que necesite la aplicación.

Tan pronto como se ejecuta el script, un servidor local de Streamlit se inicia con el cuadernillo. La aplicación es como un lienzo, donde se pueden dibujar gráficos, texto, widgets, tablas, etc.
Streamlit dispone de distintos métodos o funciones que permiten hacerlo, como por ejemplo, st.text() que escribe texto sin formato en la aplicación, y st.line_chart() que dibuja un gráfico de líneas.

La arquitectura de Streamlit permite escribir aplicaciones de la misma manera en que se escriben scripts en Python. Para esto, las aplicaciones de Streamlit tienen un flujo de datos único: cada vez que algo debe actualizarse en la pantalla, Streamlit vuelve a ejecutar todo el script de Python de principio a fin.

Esto puede ocurrir en dos situaciones:

- Cuando se modifica el código fuente de la aplicación.

- Cuando un usuario interactúa con los widgets en la aplicación. Por ejemplo, al arrastrar un deslizador, ingresar texto en un cuadro de entrada o hacer clic en un botón.

Veamos cómo funcionan algunas de estas opciones, tomando como base algunas funcionalidades del programa [youtube_detector](https://github.com/amalvarezme/SenalesSistemas/blob/master/3_SerieyTransformadaFourier/youtube_detector.ipynb).

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib

Ponemos un título a la aplicación y página.

In [ ]:
st.title("YouTube Audio Analysis and Classification")

Podemos dejar la opción para que el usuario suba un archivo, en este caso con el formato específico .xlsx

In [ ]:
uploaded_file = st.file_uploader("Upload your Excel file", type=["xlsx"])

Damos un mensaje de confirmación del archivo subido, y lo leemos con pandas.

In [ ]:
if uploaded_file:
    df = pd.read_excel(uploaded_file)
    st.write("Uploaded data:", df)

Para el momento en el que queramos identificar una canción, podemos poner la opción de que el usuario introduza el link de esa canción.

In [ ]:
custom_url = st.text_input("Enter a new YouTube URL to process:", "")

### Procesamiento de Datos
Ahora, una vez que se han introducido los datos, ocurre algo particular. La lista de canciones del archivo de excel van a utilizarse para entrenar un modelo sencillo que permita posteriormente hacer la clasificación. Como ya se mencionó antes, lo que hace streamlit es correr el script completo cada vez que haya un cambio. Para este caso en el que estamos entrenando un modelo, donde queremos que los resultados no varíen mucho, pero en especial cuyo costo computacional dependiendo del problema puede ser relativamente alto, existe una opción que nos permite guardar en caché elementos o partes (funciones) del script, que solo correrán una vez cuando se inicie y llame por primera vez, y no volverán a hacerlo a menos de que sea completamente necesario (han cambiado las condiciones o argumentos de la función).

Para evitar cálculos redundantes, Streamlit proporciona decoradores de caché como @st.cache_data y @st.cache_resource. Estos decoradores almacenan los resultados de cálculos costosos, los cuales pueden reutilizarse en diferentes ejecuciones de la aplicación.

- st.cache_data es la forma recomendada para almacenar en caché cálculos que devuelven datos o un objeto de datos serializable (por ejemplo, str, int, float, DataFrame, dict, list). Crea una nueva copia de los datos en cada llamada a la función, lo que lo hace seguro contra *mutaciones y condiciones de carrera*.

- st.cache_resource es la forma recomendada para almacenar en caché recursos globales como modelos de aprendizaje automático o conexiones a bases de datos. Usa st.cache_resource cuando tu función devuelve objetos no serializables que no quieres cargar múltiples veces. Devuelve el objeto en caché en sí, el cual se comparte entre todas las ejecuciones y sesiones sin copia ni duplicación. Si mutas un objeto que está en caché utilizando st.cache_resource, esa mutación existirá en todas las ejecuciones y sesiones.

Al agregar cálculos de larga duración a una aplicación, se puede usar st.progress() para mostrar el estado en tiempo real.

In [ ]:
@st.cache_data
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    video_info = youtube_dl.YoutubeDL().extract_info(url = video_url,download=False)
    filename = f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete... {}".format(filename))

@st.cache_resource
def load_and_process_audio_files(path='results/'):
    # Function to train the model from the Excel file
    pass

También existen otros elementos algo más interactivos (widgets) para el usuario que pueden cumplir funciones importantes, como st.button(), st.slider() o st.selectbox(). Estos se tratan como si fueran simplemente variables.

In [ ]:
if st.button("Download and Process Audio"):
    path = process_audio_from_link(df)
    x_t, label, name_c, fs = load_and_process_audio_files(path)

In [ ]:
song_duration = st.slider('Playtime for the song')

### Visualización de Datos
Streamlit brinda cierta flexibilidad y opciones para visualizar datos, bien sea con funciones propias o su integración con pyplot.

In [ ]:
st.write("Time Domain Signals")
st.line_chart(x_t.mean(axis=-1)) # Gráfico en el tiempo

In [ ]:
plt.plot(vf, abs(Xw).T)
plt.xlabel('Frequency (Hz)')
plt.ylabel('|X(f)|')
st.pyplot(plt)

Streamlit en ocasiones recomienda usar st.write() gracias a su funcionalidad "Magic". Podemos pasarle casi cualquier cosa a st.write() y automáticamente encontrará la mejor manera de representar esos datos (texto, datos, figuras, etc). Sin embargo, para un mayor control, pueden usarse los métodos mencionados antes.

### Presentación del dashboard

Streamlit facilita la organización de los widgets en un panel lateral izquierdo con st.sidebar(). Cada elemento que se pasa a st.sidebar() queda fijado a la izquierda, permitiendo a los usuarios concentrarse en el contenido de la aplicación mientras siguen teniendo acceso a los controles de la interfaz.

In [ ]:
# Add a selectbox to the sidebar:
add_selectbox = st.sidebar.selectbox(
    'How would you like to be contacted?',
    ('Email', 'Home phone', 'Mobile phone')
)

Más allá del panel lateral, Streamlit ofrece varias otras formas de controlar la disposición de tu aplicación. st.columns() permite colocar widgets uno al lado del otro, y st.expander() permite ahorrar espacio al ocultar contenido extenso.

A medida que las aplicaciones crecen, resulta útil organizarlas en múltiples páginas. Esto facilita la gestión de la aplicación y la navegación como usuario. Streamlit ofrece una forma sencilla de crear aplicaciones multipágina.
- En la carpeta donde está el script principal, se crea una nueva carpeta para las páginas [pages].
- Se añaden nuevos archivos .py en esta carpeta para añadir más páginas a la app.
- Se corre sl script principal de la misma manera.

A continuación se muestra una app básica con algunas de las funcionalidades presentadas, para visualizarlas en la app.

In [ ]:
%%writefile detector.py

import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib

st.title("YouTube Audio Analysis and Classification")
uploaded_file = st.file_uploader("Upload your Excel file", type=["xlsx"])

if uploaded_file:
    df = pd.read_excel(uploaded_file)
    st.write("Uploaded data:", df)

custom_url = st.text_input("Enter a new YouTube URL to process:", "")

song_duration = st.slider('Playtime for the song')

add_selectbox = st.sidebar.selectbox(
    'How many 5s pieces would you like to process?',
    ('1', '2', '3')
)

Writing detector.py


Para mayor detalle en el uso de streamlit, podemos recurrir a su propia documentación, con ejemplos y otros detalles: https://docs.streamlit.io/get-started